# Implementation of Anomaly detection using Autoencoders
Dataset used here is Credit Card Fraud Detection from Kaggle.

### Import required libraries

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,normalize, MinMaxScaler
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score, precision_score
from keras import backend as K
print("-------------------------------------------")
print("GPU available: ", tf.config.list_physical_devices('GPU'))
print("Keras backend: ", K.backend())
print("-------------------------------------------")

# Load layers from keras
from keras.layers import Dense, Input, Concatenate, Flatten, BatchNormalization, Dropout, LeakyReLU
from keras.models import Sequential, Model
from keras.losses import binary_crossentropy
from Disco_tf import distance_corr
from keras.optimizers import Adam
from sklearn.metrics import roc_auc_score
RANDOM_SEED = 2021 
TEST_PCT = 0.3
LABELS = ["Normal","Fraud"]

-------------------------------------------
GPU available:  []
Keras backend:  tensorflow
-------------------------------------------


In [4]:
#input Layer
input_layer = tf.keras.layers.Input(shape=(input_dim, ))
#Encoder
encoder = tf.keras.layers.Dense(encoding_dim, activation="tanh",                                
activity_regularizer=tf.keras.regularizers.l2(learning_rate))(input_layer)
encoder=tf.keras.layers.Dropout(0.2)(encoder)
encoder = tf.keras.layers.Dense(hidden_dim_1, activation='relu')(encoder)
encoder = tf.keras.layers.Dense(hidden_dim_2, activation=tf.nn.leaky_relu)(encoder)
# Decoder
decoder = tf.keras.layers.Dense(hidden_dim_1, activation='relu')(encoder)
decoder=tf.keras.layers.Dropout(0.2)(decoder)
decoder = tf.keras.layers.Dense(encoding_dim, activation='relu')(decoder)
decoder = tf.keras.layers.Dense(input_dim, activation='tanh')(decoder)
#Autoencoder
autoencoder = tf.keras.Model(inputs=input_layer, outputs=decoder)
autoencoder.summary()

NameError: name 'input_dim' is not defined

In [8]:
# build one block for each dense layer
def get_block(L, size):
    L = BatchNormalization()(L)

    L = Dense(size)(L)
    L = Dropout(0.5)(L)
    L = LeakyReLU(0.2)(L)
    return L

# baseline correlation function
def binary_cross_entropy(y_true, y_pred):
    
    return binary_crossentropy(y_true, y_pred)

# define new loss with distance decorrelation
def decorr(var_1, var_2, weights,kappa):

    def loss(y_true, y_pred):
        #return binary_crossentropy(y_true, y_pred) + distance_corr(var_1, var_2, weights)
        #return distance_corr(var_1, var_2, weights)
        return binary_crossentropy(y_true, y_pred) + kappa * distance_corr(var_1, var_2, weights)
        #return binary_crossentropy(y_true, y_pred)

    return loss

In [9]:
allX = { feat : np.genfromtxt('%s' % feat,delimiter = ',')[1:522467,:] for feat in ["Input_Background_1.csv",
                                                       "Input_Signal_1.csv"] }

In [12]:
X = list(allX.values())
y = np.ones((522466))

y[0:2000] = 0

from sklearn.model_selection import train_test_split
split = train_test_split(*X,y , test_size=0.1, random_state=42)
train = [ split[ix] for ix in range(0,len(split),2) ]
test = [ split[ix] for ix in range(1,len(split),2) ]
X_train, y_train = train[0:2], train[-1]
X_test, y_test = test[0:2], test[-1]

X_train.append(np.ones(len(y_train)))
X_test.append(np.ones(len(y_train)))

# Setup network
# make inputs
jets = Input(shape=X_train[0].shape[1:])
f_jets = Flatten()(jets)
leps = Input(shape=X_train[1].shape[1:])
f_leps = Flatten()(leps)
i = Concatenate(axis=-1)([f_jets, f_leps])
sample_weights = Input(shape=(1,))
#setup trainable layers
d1 = get_block(i, 1024)
d2 = get_block(d1, 1024)
d3 = get_block(d2, 512)
d4 = get_block(d3, 256)
d5 = get_block(d4, 128)
o = Dense(1, activation="sigmoid")(d5)

model = Model(inputs=[jets,leps, sample_weights], outputs=o)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 21)]         0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, 21)]         0                                            
__________________________________________________________________________________________________
flatten_4 (Flatten)             (None, 21)           0           input_7[0][0]                    
__________________________________________________________________________________________________
flatten_5 (Flatten)             (None, 21)           0           input_8[0][0]                    
____________________________________________________________________________________________

In [15]:
# Compile model
from keras.optimizers import Adam
opt = Adam(lr=0.001)
model.compile(optimizer=opt, loss=decorr(jets[:,0], o[:,0], sample_weights[:,0],0.5))
#model.compile(optimizer=opt, loss="binary_crossentropy")

In [ ]:
# Train model
model.fit(x=X_train, y=y_train, epochs=20, batch_size=10000, validation_split=0.1)

# Evaluate model
y_train_predict = model.predict(X_train, batch_size=10000)
y_test_predict = model.predict(X_test, batch_size=10000)
from sklearn.metrics import roc_auc_score
auc_train = roc_auc_score(y_train, y_train_predict)
auc_test = roc_auc_score(y_test, y_test_predict)
print("area under ROC curve (train sample): ", auc_train)
print("area under ROC curve (test sample): ", auc_test)

# plot correlation
x = X_test[0][:,0,0]
y = y_test_predict[:,0]
corr = np.corrcoef(x, y)
print("correlation ", corr[0][1])

Train on 423197 samples, validate on 47022 samples
Epoch 1/20


In [7]:
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()